In [1]:
#import libraries
import numpy as np
import pandas as pd
import json
import requests
from bs4 import BeautifulSoup
from selenium import webdriver

print('Libraries Imported.')

Libraries Imported.


In [2]:
#retrieve html of target
url = 'https://www.echo-usa.com/Support-Help/Parts-Lookup'

# session = HTMLSession()
# resp =  session.get(url)
# print(resp.html.html)
# # resp.html.render()

options = webdriver.ChromeOptions() 
options.add_argument('--headless') 
browser = webdriver.Chrome(executable_path='C:\Program Files\chromedriver\chromedriver.exe', 
                            options=options)
browser.get(url)
# cur_url =  browser.execute_script('return document.URL')
# print(cur_url)
# browser.get(cur_url)

#product_line = browser.find_element_by_xpath('//*[@id="Tab2"]').click()
# product_line = browser.find_element_by_xpath('//*[@id="Tab2"]')
# browser.execute_script('https://echoinc.partsmartweb.com/scripts/EmpartISAPI.dll?MF')
#product_line =  browser.find_element_by_id("Tab2")


#print(browser.page_source)
frameset = browser.find_element_by_tag_name('frameset')
print(frameset)
frame = frameset.find_element_by_tag_name('frame')
print(frame.get_attribute('src'))

browser.switch_to.frame(frame)
product_line =  browser.find_element_by_id("Tab2")
product_line.click()

<selenium.webdriver.remote.webelement.WebElement (session="adc9a030fc1f33f479b47d5cb01fa246", element="4bea40b4-bde4-43eb-bc13-114e9c80c934")>
https://echoinc.partsmartweb.com/scripts/EmpartISAPI.dll?MF&app=echo&session=904f512b-8b59-4288-b0ff-773cfdc20fc9&mode=12&TF=ewMain


In [ ]:
soup = BeautifulSoup(browser.page_source, "html.parser")
# soup = BeautifulSoup(resp.html.html, "html.parser")
print(soup)